In [1]:
!pip install -U predibase --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 2

## Initialize Predibase Client

In [9]:
from predibase import Predibase, FinetuningConfig, DeploymentConfig

pb = Predibase(api_token="pb_Ig-6eXN0WzCe9NH0dD-vTQ")

Connected to Predibase as User(id=84b37d8c-79bd-4164-ac22-f88f49dd93e7, username=sandeep.samant@rwth-aachen.de)

In [16]:
from predibase import PredibaseClient
pc = PredibaseClient(token = "pb_Ig-6eXN0WzCe9NH0dD-vTQ")

Connected to Predibase as User(id=84b37d8c-79bd-4164-ac22-f88f49dd93e7, username=sandeep.samant@rwth-aachen.de)

## connecting dataset

In [19]:
dataset = pb.datasets.from_file("final_data_itsm.csv", name="itsm_dataset")

## Creating Baseline model and finetuning it

In [20]:
# Create an adapter repository
repo = pb.repos.create(name="itsm-tasks-model", description="AI-powered ITSM troubleshooting agent ", exists_ok=True)

# Start a fine-tuning job, blocks until training is finished
adapter = pb.adapters.create(
    config=FinetuningConfig(
        base_model="llama-2-7b"
    ),
    dataset=dataset, # Also accepts the dataset name as a string
    repo=repo,
    description="initial model with defaults"
)

Successfully requested finetuning of llama-2-7b as `itsm-tasks-model/1`. (Job UUID: 2c9f5d66-6ad4-4722-b23f-0abfcd9a2762).

Watching progress of finetuning job 2c9f5d66-6ad4-4722-b23f-0abfcd9a2762. This call will block until the job has finished. Canceling or terminating this call will NOT cancel or terminate the job itself.

Job is starting. Total queue time: 0:00:47         
Waiting to receive training metrics...

┌────────────┬────────────┬─────────────────┐
│ checkpoint │ train_loss │ validation_loss │
├────────────┼────────────┼─────────────────┤
│     1      │   0.9015   │        --       │
│     2      │   0.5767   │        --       │
│     3      │   0.5993   │        --       │
└────────────┴────────────┴─────────────────┘


## Instructional tuning LLama2

In [21]:
# Create an adapter repository
repo = pb.repos.create(name="itsm-tasks-model_v2", description="AI-powered ITSM troubleshooting agent", exists_ok=True)

# Start a fine-tuning job with custom parameters, blocks until training is finished
adapter = pb.adapters.create(
    config=FinetuningConfig(
         base_model="llama-2-7b",
        epochs=1, # default: 3
        rank=8, # default: 16
        learning_rate=0.0001 # default: 0.0002
    ),
    dataset=dataset,
    repo=repo,
    description="changing epochs, rank, and learning rate"
)

Successfully requested finetuning of llama-2-7b as `itsm-tasks-model_v2/1`. (Job UUID: d6147f31-7c47-45bf-8baa-b9201ef51ada).

Watching progress of finetuning job d6147f31-7c47-45bf-8baa-b9201ef51ada. This call will block until the job has finished. Canceling or terminating this call will NOT cancel or terminate the job itself.

Job is starting. Total queue time: 0:04:46         
Waiting to receive training metrics...

┌────────────┬────────────┬─────────────────┐
│ checkpoint │ train_loss │ validation_loss │
├────────────┼────────────┼─────────────────┤
│     1      │   1.4055   │        --       │
└────────────┴────────────┴─────────────────┘


In [22]:
adapter = pb.adapters.get("itsm-tasks-model_v2/1")
adapter

Adapter(repo='itsm-tasks-model_v2', tag=1, base_model='llama-2-7b', description='changing epochs, rank, and learning rate', artifact_path='d6147f31-7c47-45bf-8baa-b9201ef51ada/4c56c79f4a234cd4853255ee05e7a5ad/artifacts/model/model_weights', finetuning_error=None, finetuning_job_uuid='d6147f31-7c47-45bf-8baa-b9201ef51ada')

In [ ]:
pb.adapters.download("itsm-tasks-model_v2/1")